In [84]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [85]:
import os
import sys
import pandas as pd
import requests
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

BASE_URL = "https://api.ravelry.com"

# The os.getenv() calls will now find the variables loaded from your .env file
RAVELRY_ACCESS_KEY = os.getenv('RAVELRY_ACCESS_KEY')
RAVELRY_PERSONAL_KEY = os.getenv('RAVELRY_PERSONAL_KEY')

In [86]:
# --- ADD THIS DEBUGGING CODE ---
print(f"Access Key Loaded: {RAVELRY_ACCESS_KEY}")
print(f"Personal Key Loaded: {RAVELRY_PERSONAL_KEY}")

Access Key Loaded: read-d4086974ad193fe02828dd97c21b9560
Personal Key Loaded: Eq5JjrVDcMu4Ji01Y2aQ9bMh4gtUpr1JoSYsG7Ri


Writing a function to test grabbing different attributes that may be useful in analysis:

In [87]:
def search_patterns(query):
    """
    Searches for patterns on Ravelry with a specific query.

    Args:
        query (str): The search term for patterns (e.g., "sweater").

    Returns:
        list: A list of patterns that match the criteria.
    """
    endpoint = f"{BASE_URL}/patterns/search.json"
    
    # Parameters for the API request
    params = {
        "query": query,
        "page_size": 5  # You can adjust the number of results per page
    }

    try:
        response = requests.get(endpoint, auth=(RAVELRY_ACCESS_KEY, RAVELRY_PERSONAL_KEY), params=params)
        response.raise_for_status()  # Raise an exception for bad status codes

        patterns = response.json()
        patterns = patterns.get('patterns', [])
        # Return all patterns from the response

        patterns_data = []
        for pattern in patterns:
            patterns_data.append({
                'Name': pattern.get('name'),
                'Photo': pattern.get('first_photo', {}).get('medium2_url'),
                'Designer': pattern.get('designer', {}).get('name'),
                'ID': pattern.get('id'),
                'URL': f"https://www.ravelry.com/patterns/library/{pattern.get('permalink')}",
                'Free' : pattern.get('free')
            })

        df = pd.DataFrame(patterns_data)
        return df  

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [89]:
def extract_categories(category_data):
    """
    Extracts all parent categories from a potentially nested category structure.
    """
    all_categories = set()
    
    # --- START: ADD THIS NEW LOGIC ---
    
    # 1. Check if the input is a list and not empty
    if isinstance(category_data, list) and category_data:
        # If it is, use the first item in the list as our starting point
        current_level = category_data[0]
    # 2. Check if the input is already a dictionary
    elif isinstance(category_data, dict):
        # If so, use it directly
        current_level = category_data
    # 3. Otherwise, we can't process it
    else:
        return [] # Return an empty list if data is invalid
        
    # --- END: NEW LOGIC ---
    
    # Your existing, corrected loop will now work correctly
    while isinstance(current_level, dict):
        if 'name' in current_level:
            all_categories.add(current_level['name'])
        current_level = current_level.get('parent')
        
    return list(all_categories)

In [90]:
def extract_permalinks(attribute_list):
    """
    Extracts the 'permalink' value from each dictionary in a list.
    """
    if not isinstance(attribute_list, list):
        return [] # Return empty list if input is not a list
        
    return [item['permalink'] for item in attribute_list if 'permalink' in item]

In [96]:
def get_pattern_details(pattern_id):
    """
    Fetches the craft, category, and attributes for a given pattern ID
    in a single API call. Returns a dictionary with the results.
    """
    details_url = f"https://api.ravelry.com/patterns/{pattern_id}.json"
    
    try:
        response = requests.get(details_url, auth=(RAVELRY_ACCESS_KEY, RAVELRY_PERSONAL_KEY))
        # Raise an exception for bad status codes (4xx or 5xx)
        response.raise_for_status()
        
        details_data = response.json()
        pattern_data = details_data.get('pattern', {}) # Use .get() for safety
        
        # Extract all needed data, providing default values
        craft = pattern_data.get('craft', [])['name']
        categories = pattern_data.get('pattern_categories', [])
        attributes = pattern_data.get('pattern_attributes', [])
        difficulty_average = pattern_data.get('difficulty_average', None)
        downloadable = pattern_data.get('downloadable', False)
        gauge = pattern_data.get('gauge', None)
        gauge_divisor = pattern_data.get('gauge_divisor', None)
        gauge_pattern = pattern_data.get('gauge_pattern', None)
        pattern_type	 = pattern_data.get('pattern_type', None)['permalink']
        yarn_weight = pattern_data.get('yarn_weight', None)['name']
        photos = pattern_data.get('photos', [])
        projects_count	 = pattern_data.get('projects_count', 0)
        rating_average = pattern_data.get('rating_average', None)
        sizes_available = pattern_data.get('sizes_available', "")

        # Return a dictionary containing all the extracted data
        return {'Craft': craft, 'Categories': extract_categories(categories), 'Attributes': extract_permalinks(attributes), 'Gauge': gauge, 'Difficulty Average': difficulty_average, 'Downloadable': downloadable, 'Gauge Divisor': gauge_divisor, 'Gauge Pattern': gauge_pattern, 'Pattern Type': pattern_type, 'Yarn Weight': yarn_weight, 'Photos': photos, 'Projects Count': projects_count, 'Rating Average': rating_average, 'Sizes Available': sizes_available}

    except requests.exceptions.RequestException as e:
        # Handle network errors or bad responses gracefully
        print(f"Could not fetch data for pattern ID {pattern_id}: {e}")
        # Return a dictionary with default empty values to prevent errors later
        return {'Craft': None, 'Categories': [], 'Attributes': [], 'Gauge': None, 'Difficulty Average': None, 'Downloadable': False, 'Gauge Description': None, 'Gauge Divisor': None, 'Gauge Pattern': None, 'Pattern Needle Sizes': "", 'Pattern Type': None, 'Yarn Weight': None, 'Photos': [], 'Projects Count': 0, 'Rating Average': None, 'Sizes Available': ""}


def add_details_to_df(df):
    """
    Applies the single, efficient function to the DataFrame to create
    three new columns from the returned data.
    """
    # Using .apply with a lambda function that returns a pandas Series
    # is an efficient way to create multiple columns at once.
    details = df['ID'].apply(lambda pid: pd.Series(get_pattern_details(pid)))
    
    # Join the newly created columns back to the original DataFrame
    return df.join(details)

In [97]:
test_sweaters = search_patterns("cardigan sweater")

In [98]:
test_sweaters_with_details = add_details_to_df(test_sweaters)

In [99]:
test_sweaters_with_details

,Name,Photo,Designer,ID,URL,Free,Craft,Categories,Attributes,Gauge,Difficulty Average,Downloadable,Gauge Divisor,Gauge Pattern,Pattern Type,Yarn Weight,Photos,Projects Count,Rating Average,Sizes Available
0,Marguerite Cardigan,https://images4-a.ravelrycache.com/uploads/Mar...,Marci Marra,7460596,https://www.ravelry.com/patterns/library/margu...,True,Knitting,"[Clothing, Sweater, Categories, Cardigan]","[lace, one-piece, top-down, chart, written-pat...",18.0,0.000000,True,4,lace,cardigan,Lace,"[{'id': 141596790, 'sort_order': 1, 'user_id':...",2,0.000000,"Sizes: 1, 2, 3, 4"
1,Columbia Cardigan,https://images4-a.ravelrycache.com/uploads/kat...,Courtney Kelley,7461129,https://www.ravelry.com/patterns/library/colum...,False,Knitting,"[Clothing, Sweater, Categories, Cardigan]","[ribbed, seamed, bottom-up, worked-flat, twist...",14.0,0.000000,False,4,Brioche Rib,cardigan,Worsted,"[{'id': 141644452, 'sort_order': 1, 'user_id':...",0,0.000000,"39¾ (44, 47¼, 50½) (56¼, 60½, 64¾)"" (101 [112,..."
2,Cardigan No. 9,https://images4-a.ravelrycache.com/uploads/MFT...,My Favourite Things,7326364,https://www.ravelry.com/patterns/library/cardi...,False,Knitting,"[Clothing, Sweater, Categories, Cardigan]","[female, unisex, adult, oversized, crew-neck, ...",16.0,3.881188,True,4,Stockinette,cardigan,Aran,"[{'id': 128737580, 'sort_order': 1, 'user_id':...",1075,4.660465,XS-4XL
3,Cucumber Cardigan,https://images4-a.ravelrycache.com/uploads/BYC...,EunMi Jung,7461266,https://www.ravelry.com/patterns/library/cucum...,False,Knitting,"[Clothing, Sweater, Categories, Cardigan]","[cables, stripes, buttoned, seamless, top-down...",33.0,0.000000,True,4,24 sets *34 rows after blocking,cardigan,Fingering,"[{'id': 141655910, 'sort_order': 1, 'user_id':...",4,0.000000,Sizes 1 (2) 3 (4) : Bust circumference 85 (90)...
4,Step by Step Cardigan,https://images4-a.ravelrycache.com/uploads/flr...,Florence Miller,7331372,https://www.ravelry.com/patterns/library/step-...,False,Knitting,"[Clothing, Sweater, Categories, Cardigan]","[female, v-neck, seamless, top-down, worked-fl...",16.0,2.884058,True,4,stockinette,cardigan,Worsted,"[{'id': 129234437, 'sort_order': 1, 'user_id':...",1735,4.901149,"(A, B, C) (D, E, F) (G, H, I)"


In [95]:
test_sweaters_with_details.to_csv("test_sweaters_with_details.csv", index=False)